In [1]:
import pandas as pd
import numpy as np
from astropy.utils.console import ProgressBar

from tqdm import tqdm_notebook

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from music.models import MusicGadgetIc, MusicEllipsoidRegion
from rockstar.models import RockstarCatalogue
from core.models import EllipsoidRegion, Category

In [3]:
from core.utils import create_ellipsoid_region

In [4]:
from music.utils import create_ic_from_existing

In [5]:
# df_selected = pd.read_hdf('selected.hdf5', key='2017-07-20 19:24:04.184707')
# df_selected = pd.read_hdf('../media/selected.hdf5', key='2017-07-24 20:16:20.495943')
df_selected = pd.read_hdf('../media/selected.hdf5', key='selection3')

In [6]:
sim = Simulation.objects.get(name='out512')
snap = sim.snapshot_set.get(snap_number=14)
cat = snap.catalogue_set.get()
halos = cat.structure_set
ic = sim.ic

In [8]:
# category = Category.objects.get(name='selection 2')

cat_name = 'Lim_4Rvir'

category = Category.objects.get(name=cat_name)

In [9]:
selected_ids = df_selected.index[:]
selected_ids = [33306, 136254]
len(selected_ids)

2

In [11]:
# create regions

# rtb = 10
# rtb = 5
# rtb = 2
rtb = 4

for hid in tqdm_notebook(selected_ids):
    structure = halos.get(rockstarhalo__hid=hid)

    # create EllipsoidRegion
    region = create_ellipsoid_region(structure, rtb=rtb)
    region.category.add(category)
    region.save()
    
    # region = regions.get(structure=structure)
        
    # create MusicEllipsoidRegion
    region = MusicEllipsoidRegion(region=region)
    region.save()

A Jupyter Widget

In [12]:
halo_ids = df_selected.index
# halo_ids = halo_ids.drop([33306, 124429])
halo_ids = [33306, 136254]


In [13]:
# category = Category.objects.get(name='selection 2')
# category = Category.objects.get(name='Lim_5Rvir')
category = Category.objects.get(name=cat_name)

level = 12
if level>=12:
    spreadcoarse = False
    post_action = 'python $HOME/wmmw/initial_conditions/spread.py *.dat'
else:
    spreadcoarse = True

with ProgressBar(len(halo_ids), ipython_widget=True) as pbar:
    for hid in halo_ids:
        
        # category = root_cat.children.get(name=str(hid))
        # category = category.children.get(name='DM')
        
        new_ic = create_ic_from_existing(ic)
        
        structure = halos.get(rockstarhalo__hid=hid)
        region = Region.objects.get(structure=structure, category=category)
        region = MusicEllipsoidRegion.objects.get(region=region)
        
        new_ic.region = region
        # new_ic.levelmin = 8
        new_ic.levelmin = 7
        new_ic.levelmax = level
        
        new_ic.padding = 6
        new_ic.overlap = 4
        
        new_ic.gadget_spreadcoarse = spreadcoarse
        new_ic.post_action = post_action
        new_ic.category.add(category)
        new_ic.name = '{hid:d}_DM_L{level:d}'.format(hid=hid, level=level)
        new_ic.filename = new_ic.name + '.dat'
        
        new_ic.save()
        
        pbar.update()

A Jupyter Widget

In [ ]:
gr.